# Programming Assignment 5.1

In [1]:
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import pandas 
import scipy

Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv

In [47]:
data_train = pandas.read_csv('salary_train.csv')
data_test = pandas.read_csv('salary_test.csv')
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


Приведите тексты к нижнему регистру (text.lower()).

In [31]:
data_train['FullDescription'] = data_train['FullDescription'].str.lower()
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london ****k ****...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager// luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse / rgn nursing home for young...,Sutton Coldfield,NaN,20355


Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты


In [40]:
data_train['FullDescription'] = data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
len(data_train['FullDescription'] )

60000

Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [65]:
tfid = TfidfVectorizer(min_df = 5)
dat_full = np.hstack([data_train['FullDescription'], data_test['FullDescription']])
converted = tfid.fit_transform(dat_full).toarray()



In [72]:
converted_train = converted[:60000]
print converted_train.shape
converted_test = converted[60000:]
print converted_test.shape

(60000, 22879)
(2, 22879)


Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.

In [79]:
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)
data_test['LocationNormalized'].fillna('nan', inplace=True)
data_test['ContractTime'].fillna('nan', inplace=True)

Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [80]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ.shape

(2, 1766)

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.


In [82]:
features = scipy.sparse.hstack((converted_train, X_train_categ))
features_test = scipy.sparse.hstack((converted_test, X_test_categ))

Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [83]:
ridge = Ridge(alpha=1.0, random_state = 241)
ridge.fit(features, data_train['SalaryNormalized']) 

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [86]:
ridge.predict(features_test)

array([ 56549.27176831,  37200.60730212])